In [1]:
import ROOT as r
import math
import array
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.backends.backend_pdf import PdfPages
import uproot
import numpy as np
from scipy.optimize import curve_fit

#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, 'include')
sys.path.insert(0, include_dir)
#__________________________________________________

import CONFIG
import DBPARSE
from UTILITIES import *
from SIMFITS2D import DistributionFits2D
from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend

Welcome to JupyROOT 6.26/06


In [2]:

GEN2Pathp1=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN2_sbs100p_nucleon_np_model2.root"
GEN3Pathp1=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN3_sbs100p_nucleon_np_model2.root"
GEN4aPathp1=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN4a_sbs100p_nucleon_np_model2.root"
GEN4bPathp1=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN4b_sbs100p_nucleon_np_model2.root"



GEN2Pathp2=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2a/Final_data_GEN2_sbs100p_nucleon_np_model2.root"
GEN3Pathp2=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2a/Final_data_GEN3_sbs100p_nucleon_np_model2.root"
GEN4aPathp2=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2a/Final_data_GEN4a_sbs100p_nucleon_np_model2.root"
GEN4bPathp2=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2a/Final_data_GEN4b_sbs100p_nucleon_np_model2.root"

GEN3PathTiming=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_timing_GEN3_sbs100p_nucleon_np_model2.root"
GEN2PathTiming=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_timing_GEN2_sbs100p_nucleon_np_model2.root"

GEN3PathTiming=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_timing_GEN3_sbs100p_nucleon_np_model2.root"
GEN2PathTiming=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_timing_GEN2_sbs100p_nucleon_np_model2.root"
GEN4aPathTiming=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_timing_GEN4a_sbs100p_nucleon_np_model2.root"
GEN4bPathTiming=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_timing_GEN4b_sbs100p_nucleon_np_model2.root"

GEN3InSim=f"/media/research/TOSHIBA EXT/GEn/outfiles/insim/QE_Insim_GEN3_sbs100p_nucleon_np_model2.root"
GEN3InSimNEW=f"/media/research/TOSHIBA EXT/GEn/outfiles/insim/newprobsamp/QE_Insim_GEN3_sbs100p_nucleon_np_model2.root"
GEN2InSim=f"/media/research/TOSHIBA EXT/GEn/outfiles/insim/QE_Insim_GEN2_sbs100p_nucleon_np_model2.root"
GEN4InSim=f"/media/research/TOSHIBA EXT/GEn/outfiles/insim/QE_Insim_GEN4_sbs100p_nucleon_np_model2.root"


GEN2InSimNEW=f"/media/research/TOSHIBA EXT/GEn/outfiles/insim/newprobsamp/QE_Insim_GEN2_sbs100p_nucleon_np_model2.root"
GEN4InSimNEW=f"/media/research/TOSHIBA EXT/GEn/outfiles/insim/newprobsamp/QE_Insim_GEN4_sbs100p_nucleon_np_model2.root"




In [3]:
#available branches
import ROOT as r


def list_branches(rootfile_path, tree_name="Tout"):
    # Open the ROOT file
    root_file = r.TFile.Open(rootfile_path)
    
    # Check if the file was successfully opened
    if not root_file or root_file.IsZombie():
        print(f"Error: Could not open ROOT file '{rootfile_path}'.")
        return
    
    # Access the TTree
    tree = root_file.Get(tree_name)
    
    # Check if the tree exists
    if not tree:
        print(f"Error: Tree '{tree_name}' not found in file '{rootfile_path}'.")
        root_file.Close()
        return
    
    # Get the list of branches
    branch_list = tree.GetListOfBranches()
    
    # Print all branch names
    print(f"Branches in tree '{tree_name}':")
    for branch in branch_list:
        print(branch.GetName())
    
    # Close the ROOT file
    root_file.Close()

# Example usage
list_branches(GEN4InSimNEW)


Branches in tree 'Tout':
WCut
pCut
nCut
weight
mc_Aperp1
mc_sig
mc_sigOld
mc_sigPol
mc_BETA
mc_THETA
fnucl
fiduCut
coinCut
ebeam
nu
Q2
W2
dpel
ephi
etheta
pcentral
vz
vx
vy
xtgt
ytgt
thtgt
phtgt
xfp
yfp
thfp
phfp
trP
trPx
trPy
trPz
trX
trY
trTh
trPh
ePS
eSH
eHCAL
xHCAL
yHCAL
xHCAL_exp
yHCAL_exp
dx
dy
grinch_track
grinch_clus_size


Error in <TList::Clear>: A list is accessing an object (0x34577560) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x34642570) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x346429a0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x346895e0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x34689950) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x34689cc0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x3468a1f0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x346a7a60) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x346af050) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x346af3

In [3]:
"""
#inputs hodo
config="3"
prefix="HCal_data"
detector="hodo"
#inputs hcal
config="3"
prefix="HCal_data"
detector="hcal"
"""

'\n#inputs hodo\nconfig="3"\nprefix="HCal_data"\ndetector="hodo"\n#inputs hcal\nconfig="3"\nprefix="HCal_data"\ndetector="hcal"\n'

In [12]:
#define branches
branches=["ebeam","nu","Q2","W2","dpel","ephi","etheta","pcentral","vz","vx"
          ,"vy","xtgt","ytgt","thtgt","phtgt","thetabend","xfp","yfp","thfp"
          ,"phfp","trP","trPx","trPy","trPz","ePS","xPS","eSH","xSH","ySH","eHCAL"
          ,"xHCAL","yHCAL","xHCAL_exp","yHCAL_exp","dx","dy","ngrinch_hits","xGRINCH"
          ,"yGRINCH","coin_time","hcal_time","bbcal_time","BPMAx","BPMAy","Rasterx"
          ,"Rastery","Raster2x","Raster2y","helicity","IHWP","pblkid","tdc","atime"
          ,"nblk","mag","runnum","tar","cblkid","cblkatime","cblktime","cblke","nclus"
          ,"cid","cx","cy","catime","hodoTimes","hodoIDs","hodoTOT","nbars"]
hodo_branches=["hodoTimes","hodoIDs","hodoTOT","nbars","hodoClusMean"]
hcal_branches=["cblkid","cblkatime","cblktime","cblke","nclus"
          ,"cid","cx","cy","catime"]
trigger_branches=["trigger","triggerID","NdataTriggerID"]
generic_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","ephi","trP","thtgt","phtgt",
                  "coin_time","trPx","trPy","trPz","ephi","ebeam","helicity","IHWP","runnum","eHCAL"]
generic_branchesp2=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","ephi","trP","thtgt","phtgt",
                  "coin_time","new_coin_time","trPx","trPy","trPz","ephi","ebeam","helicity","IHWP","runnum",
                    "grinch_clus_size",
                   "grinch_clus_trackindex","HCALtw","HODOmeantime","bb_rftime","sbs_rftime","eHCAL"]
sim_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","ephi","trP","fnucl","weight",
             "eHCAL"]
insim_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","trP","fnucl","weight","mc_sig",
               "mc_sigOld",
               "mc_sigPol",
               "mc_BETA",
               "mc_THETA",
               "eHCAL"]


ml_branches = [
    "ebeam", "nu", "Q2", "W2", "dpel", "ephi", "etheta", "pcentral", "vz", "vx", "vy",
    "xtgt", "ytgt", "thtgt", "phtgt", "xfp", "yfp", "thfp", "phfp", "trP",
    "ePS", "eSH", "eHCAL", "xHCAL", "yHCAL", "xHCAL_exp", "yHCAL_exp",
    "dx", "dy"
]
tofcal_branches = [
    "ePS",
    "runnum",
    "tleft",
    "tright",
    "totleft",
    "totright",
    "vpos",
    "barid",
    "nbars",
    "nblk",
    "bb_rftime",
    "hodo_rftime",
    "sbs_rftime",
    "hcal_rftime",
    "bb_trigtime",
    "hodo_trigtime",
    "sbs_trigtime",
    "hcal_trigtime",
    "cblke",
    "cblkid",
    "cblktime",
    "vz",
    "xHCAL",
    "yHCAL",
    "xSH",
    "ySH",
    "pathl",
    "tr_y",
    "tr_x",
    "tr_th",
    "tr_ph",
    "meantime",
    "HCALtw",
    "W2",
    "dx",
    "dy",
    "trPx",
    "trPy",
    "trPz",
    "trP",
    "cblkid",
    "nblk",
    "cblke",
    "cblktime",
    "pN_expected",
    "npathl"
    
]

tofcal_branches2 = [
    "ePS",
    "runnum",
    "nblk",
    "nblkB",
    "cblke",
    "cblkid",
    "cblktime",
    "cblkatime",
    "cblkeB",
    "cblkidB",
    "cblkatimeB",
    "vz",
    "xHCAL",
    "yHCAL",
    "xSH",
    "ySH",
    "pathl",
    "tr_y",
    "tr_x",
    "tr_th",
    "tr_ph",
    "W2",
    "dx",
    "dy",
    "trPx",
    "trPy",
    "trPz",
    "trP",
    "pN_expected",
    "npathl"
    
]

In [6]:

def ConvertToNP(branches,config,prefix,detector,File,Pass):
    # Open the ROOT file
    with uproot.open(File) as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        print(total_entries)
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)

        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")
            
            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpyData/Pass{Pass}/GEN{config}/{prefix}/{detector}{j}.npz", **data)


In [13]:
def ConvertToNPSim(branches,config,detector,N2=False):
    # Open the ROOT file
    if N2:
        string=f"/media/research/TOSHIBA EXT/GEn/outfiles/sim/QE_simN2_GEN{config}_sbs100p_nucleon_np_model2.root"
    else:
        string=f"/media/research/TOSHIBA EXT/GEn/outfiles/sim/QE_sim_GEN{config}_sbs100p_nucleon_np_model2.root"
    with uproot.open(string) as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)
        print(total_entries)
        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpySim/GEN{config}/{detector}{j}.npz", **data)


In [8]:
def ConvertToNPInSim(branches,config,prefix,detector,File):
    # Open the ROOT file
    with uproot.open(File) as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)
        print(total_entries)
        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpySim/GEN{config}/{detector}{j}.npz", **data)


# TIMING

In [11]:
ConvertToNP(tofcal_branches2,"4b","Timing","timing2_",GEN4bPathTiming,"2")

3061438
Processing Chunk 1 out of 2
Processing Chunk 2 out of 2


In [12]:
ConvertToNP(tofcal_branches2,"4a","Timing","timing2_",GEN4aPathTiming,"2")

4720157
Processing Chunk 1 out of 3
Processing Chunk 2 out of 3
Processing Chunk 3 out of 3


In [14]:
ConvertToNP(tofcal_branches2,"3","Timing","timing2_",GEN3PathTiming,"2")

30369162
Processing Chunk 1 out of 16
Processing Chunk 2 out of 16
Processing Chunk 3 out of 16
Processing Chunk 4 out of 16
Processing Chunk 5 out of 16
Processing Chunk 6 out of 16
Processing Chunk 7 out of 16
Processing Chunk 8 out of 16
Processing Chunk 9 out of 16
Processing Chunk 10 out of 16
Processing Chunk 11 out of 16
Processing Chunk 12 out of 16
Processing Chunk 13 out of 16
Processing Chunk 14 out of 16
Processing Chunk 15 out of 16
Processing Chunk 16 out of 16


In [15]:
ConvertToNP(tofcal_branches2,"2","Timing","timing2_",GEN2PathTiming,"2")

26082060
Processing Chunk 1 out of 14
Processing Chunk 2 out of 14
Processing Chunk 3 out of 14
Processing Chunk 4 out of 14
Processing Chunk 5 out of 14
Processing Chunk 6 out of 14
Processing Chunk 7 out of 14
Processing Chunk 8 out of 14
Processing Chunk 9 out of 14
Processing Chunk 10 out of 14
Processing Chunk 11 out of 14
Processing Chunk 12 out of 14
Processing Chunk 13 out of 14
Processing Chunk 14 out of 14


In [ ]:
#HODO BELOW BBCAL ABOVE

In [8]:
ConvertToNP(tofcal_branches,"3","Timing","timing",GEN3PathTiming,"2")

30506510
Processing Chunk 1 out of 16
Processing Chunk 2 out of 16
Processing Chunk 3 out of 16
Processing Chunk 4 out of 16
Processing Chunk 5 out of 16
Processing Chunk 6 out of 16
Processing Chunk 7 out of 16
Processing Chunk 8 out of 16
Processing Chunk 9 out of 16
Processing Chunk 10 out of 16
Processing Chunk 11 out of 16
Processing Chunk 12 out of 16
Processing Chunk 13 out of 16
Processing Chunk 14 out of 16
Processing Chunk 15 out of 16
Processing Chunk 16 out of 16


In [9]:
ConvertToNP(tofcal_branches,"2","Timing","timing",GEN2PathTiming,"2")

26082060
Processing Chunk 1 out of 14
Processing Chunk 2 out of 14
Processing Chunk 3 out of 14
Processing Chunk 4 out of 14
Processing Chunk 5 out of 14
Processing Chunk 6 out of 14
Processing Chunk 7 out of 14
Processing Chunk 8 out of 14
Processing Chunk 9 out of 14
Processing Chunk 10 out of 14
Processing Chunk 11 out of 14
Processing Chunk 12 out of 14
Processing Chunk 13 out of 14
Processing Chunk 14 out of 14


In [ ]:


#Hodoscope
ConvertToNP(hodo_branches,"3","HCal_data","hodo")

#HCal
ConvertToNP(hcal_branches,"3","HCal_data","hcal")

#Trigger
ConvertToNP(trigger_branches,"3","HCal_data","trigger")

# Generic Pass1

In [12]:
#Generic
ConvertToNP(generic_branches,"2","He3","generic",GEN2Pathp1,"1")

Processing Chunk 1 out of 14
Processing Chunk 2 out of 14
Processing Chunk 3 out of 14
Processing Chunk 4 out of 14
Processing Chunk 5 out of 14
Processing Chunk 6 out of 14
Processing Chunk 7 out of 14
Processing Chunk 8 out of 14
Processing Chunk 9 out of 14
Processing Chunk 10 out of 14
Processing Chunk 11 out of 14
Processing Chunk 12 out of 14
Processing Chunk 13 out of 14
Processing Chunk 14 out of 14


In [13]:
#Generic
ConvertToNP(generic_branches,"3","He3","generic",GEN3Pathp1,"1")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13


In [15]:
#Generic
ConvertToNP(generic_branches,"4a","He3","generic",GEN4aPathp1,"1")

Processing Chunk 1 out of 5
Processing Chunk 2 out of 5
Processing Chunk 3 out of 5
Processing Chunk 4 out of 5
Processing Chunk 5 out of 5


# Generic Pass 2

In [11]:
#Generic
ConvertToNP(generic_branchesp2,"2","He3","final_2_",GEN2Pathp2,"2")

35786069
Processing Chunk 1 out of 18
Processing Chunk 2 out of 18
Processing Chunk 3 out of 18
Processing Chunk 4 out of 18
Processing Chunk 5 out of 18
Processing Chunk 6 out of 18
Processing Chunk 7 out of 18
Processing Chunk 8 out of 18
Processing Chunk 9 out of 18
Processing Chunk 10 out of 18
Processing Chunk 11 out of 18
Processing Chunk 12 out of 18
Processing Chunk 13 out of 18
Processing Chunk 14 out of 18
Processing Chunk 15 out of 18
Processing Chunk 16 out of 18
Processing Chunk 17 out of 18
Processing Chunk 18 out of 18


In [12]:
#Generic
ConvertToNP(generic_branchesp2,"3","He3","final_2_",GEN3Pathp2,"2")

64732891
Processing Chunk 1 out of 33
Processing Chunk 2 out of 33
Processing Chunk 3 out of 33
Processing Chunk 4 out of 33
Processing Chunk 5 out of 33
Processing Chunk 6 out of 33
Processing Chunk 7 out of 33
Processing Chunk 8 out of 33
Processing Chunk 9 out of 33
Processing Chunk 10 out of 33
Processing Chunk 11 out of 33
Processing Chunk 12 out of 33
Processing Chunk 13 out of 33
Processing Chunk 14 out of 33
Processing Chunk 15 out of 33
Processing Chunk 16 out of 33
Processing Chunk 17 out of 33
Processing Chunk 18 out of 33
Processing Chunk 19 out of 33
Processing Chunk 20 out of 33
Processing Chunk 21 out of 33
Processing Chunk 22 out of 33
Processing Chunk 23 out of 33
Processing Chunk 24 out of 33
Processing Chunk 25 out of 33
Processing Chunk 26 out of 33
Processing Chunk 27 out of 33
Processing Chunk 28 out of 33
Processing Chunk 29 out of 33
Processing Chunk 30 out of 33
Processing Chunk 31 out of 33
Processing Chunk 32 out of 33
Processing Chunk 33 out of 33


In [13]:
#Generic
ConvertToNP(generic_branchesp2,"4a","He3","final_2_",GEN4aPathp2,"2")

19836450
Processing Chunk 1 out of 10
Processing Chunk 2 out of 10
Processing Chunk 3 out of 10
Processing Chunk 4 out of 10
Processing Chunk 5 out of 10
Processing Chunk 6 out of 10
Processing Chunk 7 out of 10
Processing Chunk 8 out of 10
Processing Chunk 9 out of 10
Processing Chunk 10 out of 10


In [14]:
ConvertToNP(generic_branchesp2,"4b","He3","final_2_",GEN4bPathp2,"2")

18684232
Processing Chunk 1 out of 10
Processing Chunk 2 out of 10
Processing Chunk 3 out of 10
Processing Chunk 4 out of 10
Processing Chunk 5 out of 10
Processing Chunk 6 out of 10
Processing Chunk 7 out of 10
Processing Chunk 8 out of 10
Processing Chunk 9 out of 10
Processing Chunk 10 out of 10


In [ ]:
#MachineLearning Data/ElasticSim/InelasticSim
ConvertToNP(ml_branches,"3","HCal_data","machinelearningD")
ConvertToNPSim(ml_branches,"3","HCal_data","machinelearningES")
ConvertToNPInSim(ml_branches,"3","HCal_data","machinelearningIS")

In [14]:
#Simulation
ConvertToNPSim(sim_branches,"2","simulation")
ConvertToNPSim(sim_branches,"3","simulation")
ConvertToNPSim(sim_branches,"4","simulation")

129704
Processing Chunk 1 out of 1
264763
Processing Chunk 1 out of 1
450806
Processing Chunk 1 out of 1


In [13]:
#Simulation N2
ConvertToNPSim(sim_branches,"2","simulationN2",True)
ConvertToNPSim(sim_branches,"3","simulationN2",True)
ConvertToNPSim(sim_branches,"4","simulationN2",True)

82850
Processing Chunk 1 out of 1
265216
Processing Chunk 1 out of 1
367394
Processing Chunk 1 out of 1


In [ ]:
ConvertToNPSim(sim_branches,"2","HCal_data","simulation",3)

In [9]:
ConvertToNPInSim(insim_branches,"3","HCal_data","insimulationNEW",GEN3InSimNEW)

13805
Processing Chunk 1 out of 1


In [10]:
ConvertToNPInSim(insim_branches,"2","HCal_data","insimulationNEW",GEN2InSimNEW)

11548
Processing Chunk 1 out of 1


In [11]:
ConvertToNPInSim(insim_branches,"4","HCal_data","insimulationNEW",GEN4InSimNEW)

45433
Processing Chunk 1 out of 1


In [19]:
#Simulation
ConvertToNPInSim(insim_branches,"3","HCal_data","insimulation",GEN3InSim)

311751
Processing Chunk 1 out of 1


In [17]:
ConvertToNPInSim(insim_branches,"2","HCal_data","insimulation",GEN2InSim)

73561
Processing Chunk 1 out of 1


In [19]:
ConvertToNPInSim(insim_branches,"4","HCal_data","insimulation",GEN4InSim)

223344
Processing Chunk 1 out of 1
